# Demo: Build AI Agent using CrewAI

**Overview**
This lab builds a 2-agent workflow with [CrewAI]:
1. Minute Taker extracts a structured JSON from a raw meeting transcript.
2. Minute Formatter turns that JSON into clean Markdown minutes.
   
We run everything locally using Ollama models via LiteLLM.

---
Quick Introduction:

**What is AI Agent?**
An AI agent is an LLM-driven program that’s given a role, goals, and (optionally) tools. It reads a task description, plans how to solve it, takes actions (e.g., call tools or other agents), and returns a result that follows a clear output contract.

**The Elements of an AI Agent (CrewAI terms)**
- Agent = Who am I? What’s my goal?
- Task = What exactly should I do now? With what inputs? Under what rules?
- Output = What format must I return?
- Crew = Run these tasks, with these agents, in this order(orchestration).
- Other useful parts:
    - Goal: High-level outcome (“parse minutes accurately”)
    - Context: Inputs or prior results passed into the task (context=[previous_task])
    - LLM / Parameters: Model choice, temperature, max_tokens
    - Tools (optional): Functions/APIs the agent can call
    - Process: How tasks run (Process.sequential, hierarchical, etc.)

In [10]:
%pip install -U crewai crewai-tools langchain langchain-ollama litellm

  Using cached litellm-1.77.4-py3-none-any.whl.metadata (42 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### 1. Setup LLM (Ollama)

In [ ]:
# Quick Check that LiteLLM + Ollama is working
import os,litellm

OLLAMA_BASE = os.getenv("OLLAMA_BASE", "http://localhost:11434")

resp = litellm.completion(
    model="ollama/llama3.2", 
    messages=[{"role":"user","content":"Say hello in 3 words."}],
    api_base=OLLAMA_BASE 
)
print(resp.choices[0].message.content)


Hello there!


>Tip: Setting both base_url and api_base avoids version differences in CrewAI/LiteLLM.

In [12]:
from crewai import LLM

OLLAMA_BASE = os.getenv("OLLAMA_BASE", "http://localhost:11434")

llm = LLM(
    model="ollama/llama3.2", 
    temperature=0.2,
    max_tokens=512,
    # Some crewAI versions read base_url, others api_base; set both to be safe:
    base_url=OLLAMA_BASE,
    api_base=OLLAMA_BASE,
)
print("CrewAI LLM ready")


CrewAI LLM ready


### 2. Sample meeting transcript

In [13]:
MEETING_RAW_DOC = """
Meeting: Product Sync — Mobile App v2.1
Date: 2025-09-24, 10:00–10:30
Attendees: Alice (PM), Ben (Eng Lead), Chloe (Design), Dan (QA)

Agenda:
1) Onboarding funnel drop-offs
2) Release v2.1 dates & scope
3) Support load after v2.0

Notes:
- Ben: Crash rate after v2.0 hotfix is 0.18%, steady.
- Chloe: New empty state mockups ready Friday.
- Alice: We must improve Step 2 completion; propose shorter profile form.
- Dan: Test coverage for payment edge cases at 76%. Needs more cases.

Decisions:
- Shorten onboarding Step 2 to a single screen (was 2). Target in v2.1.
- Keep A/B test for copy on Step 1; no change this week.

Action Items:
- Ben: Implement Step 2 consolidation. Due: 2025-10-03.
- Chloe: Share empty state assets to Eng. Due: 2025-09-26.
- Dan: Add 5 edge-case tests for payments. Due: 2025-09-27.

Risks:
- Support backlog rising; Alice to re-check staffing by next week.
"""
print("Loaded sample transcript.")


Loaded sample transcript.


### 3. Define Agents & Tasks (Extractor → Formatter)

- Agent “Minute Taker”: extracts one JSON with a fixed schema.
  - The Task description enforces “Return ONLY the JSON” to avoid extra text.

- Agent “Minute Formatter”: converts JSON into Markdown (no commentary).
  - The Task gives a section outline (Title/Date, Attendees, Agenda, …, Action Items table).

- The Crew runs both tasks sequentially so the formatter receives the extractor’s output.

In [14]:
from crewai import Agent, Task, Crew, Process

MEETING_RAW = MEETING_RAW_DOC.strip()

extractor = Agent(
    role="Minute Taker",
    goal="Parse meeting transcripts into accurate, structured minutes.",
    backstory="You are precise and avoid hallucinations. If a field is missing, use 'UNKNOWN'.",
    llm=llm,
    verbose=True,
)

formatter = Agent(
    role="Minute Formatter",
    goal="Convert JSON data to Markdown format without any commentary or explanation",
    backstory="You output only the requested format. Never add explanations or meta-commentary.",
    llm=llm,
    verbose=True,
)

task_extract = Task(
    description=(
        "From the transcript below, extract a single JSON object with this schema:\n\n"
        "{\n"
        '  "title": str,\n'
        '  "date": str,\n'
        '  "attendees": [str],\n'
        '  "agenda": [str],\n'
        '  "key_points": [str],\n'
        '  "decisions": [str],\n'
        '  "risks": [str],\n'
        '  "action_items": [\n'
        '     {"owner": str, "task": str, "due": str, "status": "OPEN"}\n'
        "  ]\n"
        "}\n\n"
        "Rules:\n"
        "- Use ONLY information in the transcript.\n"
        "- If something is not stated, set to 'UNKNOWN' (or empty list for arrays).\n"
        "- Keep action item owners and due dates exactly as written if present.\n\n"
        f"Transcript:\n{MEETING_RAW}\n\n"
        "Return ONLY the JSON (no explanation)."
    ),
    expected_output="A single JSON object following the schema.",
    agent=extractor,
)

task_format = Task(
    description=(
        "You are given a JSON of structured minutes (from the previous task). "
        "Create a concise Markdown report with sections:\n"
        "1) Title & Date\n"
        "2) Attendees (comma-separated)\n"
        "3) Agenda (bullets)\n"
        "4) Key Points (bullets)\n"
        "5) Decisions (bullets)\n"
        "6) Risks (bullets)\n"
        "7) Action Items (table: Owner | Task | Due | Status)\n\n"
        "Rules:\n- Do not invent content; use JSON fields as-is.\n- Keep it compact and readable."
    ),
    expected_output="A Markdown document.",
    agent=formatter,
    context=[task_extract],
)

crew = Crew(
    agents=[extractor, formatter],
    tasks=[task_extract, task_format],
    process=Process.sequential,
    verbose=True, 
)


### 4. Run the Workflow

In [15]:
result = crew.kickoff()
clean_minutes = result.tasks_output[-1].raw  # Get formatter output
print(clean_minutes)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: bd730eb4-f908-4aca-b648-c30e51eee555                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Minute Taker                                                                                            │
│                                                                                                                 │
│  Task: From the transcript below, extract a single JSON object with this schema:                                │
│                                                                                                                 │
│  {                                                                                                              │
│    "title": str,                                                                                                │
│    "date": str,                                                                                                 │
│    "attendees": [str],                                                                                          │
│    "agenda": [str],                                                                                             │
│    "key_points": [str],                                                                                         │
│    "decisions": [str],                                                                                          │
│    "risks": [str],                                                                                              │
│    "action_items": [                                                                                            │
│       {"owner": str, "task": str, "due": str, "status": "OPEN"}                                                 │
│    ]                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
│  Rules:                                                                                                         │
│  - Use ONLY information in the transcript.                                                                      │
│  - If something is not stated, set to 'UNKNOWN' (or empty list for arrays).                                     │
│  - Keep action item owners and due dates exactly as written if present.                                         │
│                                                                                                                 │
│  Transcript:                                                                                                    │
│  Meeting: Product Sync — Mobile App v2.1                                                                        │
│  Date: 2025-09-24, 10:00–10:30                                                                                  │
│  Attendees: Alice (PM), Ben (Eng Lead), Chloe (Design), Dan (QA)                                                │
│                                                                                                                 │
│  Agenda:                                                                                                        │
│  1) Onboarding funnel drop-offs                                                                                 │
│  2) Release v2.1 dates & scope                                                                                  │
│  3) Support load after v2.0                                                                                     │
│                                                                                                                 │
│  Notes:                                                

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Minute Taker                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {"title": "Product Sync — Mobile App v2.1", "date": "2025-09-24, 10:00–10:30", "attendees": ["Alice (PM)",     │
│  "Ben (Eng Lead)", "Chloe (Design)", "Dan (QA)"], "agenda": ["Onboarding funnel drop-offs", "Release v2.1       │
│  dates & scope", "Support load after v2.0"], "key_points": ["Crash rate after v2.0 hotfix is 0.18%", "New       │
│  empty state mockups ready Friday", "We must improve Step 2 completion; propose shorter profile form", "Test    │
│  coverage for payment edge cases at 76%. Needs more cases."], "decisions": ["Shorten onboarding Step 2 to a     │
│  single screen (was 2). Target in v2.1.", "Keep A/B test for copy on Step 1; no change this week."], "risks":   │
│  ["Support backlog rising; Alice to re-check staffing by next week."], "action_items": [{"owner": "Ben",        │
│  "task": "Implement Step 2 consolidation", "due": "2025-10-03"}, {"owner": "Chloe", "task": "Share empty state  │
│  assets to Eng", "due": "2025-09-26"}, {"owner": "Dan", "task": "Add 5 edge-case tests for payments", "due":    │
│  "2025-09-27"}]}                                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 41cbadd1-5a88-4fc4-9ede-b7ff60860933                                                                     │
│  Agent: Minute Taker                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Minute Formatter                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Product Sync — Mobile App v2.1                                                                               │
│  ## 2025-09-24, 10:00–10:30                                                                                     │
│                                                                                                                 │
│  ### Attendees                                                                                                  │
│  Alice (PM), Ben (Eng Lead), Chloe (Design), Dan (QA)                                                           │
│                                                                                                                 │
│  ### Agenda                                                                                                     │
│  * Onboarding funnel drop-offs                                                                                  │
│  * Release v2.1 dates & scope                                                                                   │
│  * Support load after v2.0                                                                                      │
│                                                                                                                 │
│  ### Key Points                                                                                                 │
│  * Crash rate after v2.0 hotfix is 0.18%                                                                        │
│  * New empty state mockups ready Friday                                                                         │
│  * We must improve Step 2 completion; propose shorter profile form                                              │
│  * Test coverage for payment edge cases at 76%. Needs more cases.                                               │
│                                                                                                                 │
│  ### Decisions                                                                                                  │
│  * Shorten onboarding Step 2 to a single screen (was 2). Target in v2.1.                                        │
│  * Keep A/B test for copy on Step 1; no change this week.                                                       │
│                                                                                                                 │
│  ### Risks                                                                                                      │
│  * Support backlog rising; Alice to re-check staffing by next week.                                             │
│                                                                                                                 │
│  ### Action Items                                                                                               │
│  | Owner | Task | Due | Status |                                                                                │
│  | --- | --- | --- | --- |                                                                                      │
│  | Ben | Implement Step 2 consolidation | 2025-10-03 | - |                                                      │
│  | Chloe | Share empty state assets to Eng | 2025-09-26 | - |                                                   │
│  | Dan | Add 5 edge-case tests for payments | 2025-09-2

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 107f52df-0cb4-4d7f-8cca-54909f1bf36c                                                                     │
│  Agent: Minute Formatter                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: bd730eb4-f908-4aca-b648-c30e51eee555                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Product Sync — Mobile App v2.1                                                                 │
│  ## 2025-09-24, 10:00–10:30                                                                                     │
│                                                                                                                 │
│  ### Attendees                                                                                                  │
│  Alice (PM), Ben (Eng Lead), Chloe (Design), Dan (QA)                                                           │
│                                                                                                                 │
│  ### Agenda                                                                                                     │
│  * Onboarding funnel drop-offs                                                                                  │
│  * Release v2.1 dates & scope                                                                                   │
│  * Support load after v2.0                                                                                      │
│                                                                                                                 │
│  ### Key Points                                                                                                 │
│  * Crash rate after v2.0 hotfix is 0.18%                                                                        │
│  * New empty state mockups ready Friday                                                                         │
│  * We must improve Step 2 completion; propose shorter profile form                                              │
│  * Test coverage for payment edge cases at 76%. Needs more cases.                                               │
│                                                                                                                 │
│  ### Decisions                                                                                                  │
│  * Shorten onboarding Step 2 to a single screen (was 2). Target in v2.1.                                        │
│  * Keep A/B test for copy on Step 1; no change this week.                                                       │
│                                                                                                                 │
│  ### Risks                                                                                                      │
│  * Support backlog rising; Alice to re-check staffing by next week.                                             │
│                                                                                                                 │
│  ### Action Items                                                                                               │
│  | Owner | Task | Due | Status |                                                                                │
│  | --- | --- | --- | --- |                                                                                      │
│  | Ben | Implement Step 2 consolidation | 2025-10-03 | - |                                                      │
│  | Chloe | Share empty state assets to Eng | 2025-09-2

# Product Sync — Mobile App v2.1
## 2025-09-24, 10:00–10:30

### Attendees
Alice (PM), Ben (Eng Lead), Chloe (Design), Dan (QA)

### Agenda
* Onboarding funnel drop-offs
* Release v2.1 dates & scope
* Support load after v2.0

### Key Points
* Crash rate after v2.0 hotfix is 0.18%
* New empty state mockups ready Friday
* We must improve Step 2 completion; propose shorter profile form
* Test coverage for payment edge cases at 76%. Needs more cases.

### Decisions
* Shorten onboarding Step 2 to a single screen (was 2). Target in v2.1.
* Keep A/B test for copy on Step 1; no change this week.

### Risks
* Support backlog rising; Alice to re-check staffing by next week.

### Action Items
| Owner | Task | Due | Status |
| --- | --- | --- | --- |
| Ben | Implement Step 2 consolidation | 2025-10-03 | - |
| Chloe | Share empty state assets to Eng | 2025-09-26 | - |
| Dan | Add 5 edge-case tests for payments | 2025-09-27 | - |


### 5. (Optional) Save to a `.md` file

In [16]:
from pathlib import Path

Path("meeting_minutes.md").write_text(clean_minutes, encoding="utf-8")
print("Saved to meeting_minutes.md")

Saved to meeting_minutes.md
